In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.io import to_json, write_json

In [6]:
px.set_mapbox_access_token(open(".mapbox_token").read())
daily_global = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/11-19-2020.csv'
df = pd.read_csv(daily_global)
df = df.dropna(axis=0, subset=['Lat'])
df = df.fillna(0)
df['Active'] = df['Active'].astype(int)
df[df['Active'] < 0] = 0
date, time = df.iloc[0,4].split()
fig = px.scatter_mapbox(df,  
                        lat="Lat", 
                        lon="Long_",     
                        color="Active", 
                        size="Active",
                        color_continuous_scale=px.colors.sequential.Jet, 
                        size_max=40, #center={"lat": 36.978, "lon": -72.249},
                        zoom=1,                 
                        hover_data=['Confirmed', 'Deaths', 'Recovered'],
                        hover_name='Combined_Key'
                       )
fig.update_layout(
    title=f'COVID-19 Global Active cases by country. Date: {date}',
)
fig.data[0].update(hovertemplate='<b>%{hovertext}</b><br><br>Active=%{marker.color}<br>Confirmed=%{customdata[0]}<br>Deaths=%{customdata[1]}<br>Recovered=%{customdata[2]}<extra></extra>')
fig.show()

In [9]:
df["Case-Fatality Ratio"] = round((df["Deaths"]/df["Confirmed"])*100, 2)

In [10]:
df["Case-Fatality Ratio"] = df["Case-Fatality Ratio"].fillna(0)

In [11]:
us_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/11-07-2020.csv')
us_df["Case-Fatality Ratio"] = (us_df["Deaths"]/us_df["Confirmed"])*100
us_df["Case-Fatality Ratio"] = us_df["Case-Fatality Ratio"].fillna(0)

In [12]:
for state in us_df.Province_State:
    df["Lat"] = np.where((df.Province_State == state),us_df[us_df["Province_State"] == state].Lat.values,df['Lat'])
    df["Long_"] = np.where((df.Province_State == state),us_df[us_df["Province_State"] == state].Long_.values,df['Long_'])
    df['Case-Fatality Ratio'] = np.where((df.Province_State == state),us_df[us_df["Province_State"] == state].iloc[:,-1].values,df['Case-Fatality Ratio'])
df = df.drop("Case_Fatality_Ratio", axis = 1)

In [13]:
new = df.Combined_Key.str.split(", ", n = 2, expand = True)
new
#new[1] = new[1].fillna(" ")
df["Province_State"].replace(0, '', inplace = True)
df['Regions'] = df[["Province_State", "Country_Region"]].astype(str).apply(lambda x: " ".join(x), axis = 1)

In [14]:
fig = px.scatter_mapbox(df,  
                        lat="Lat", 
                        lon="Long_",     
                        color="Case-Fatality Ratio", 
                        size = df["Case-Fatality Ratio"].astype(int),
                        #color_continuous_scale=px.colors.sequential.Jet, 
                        size_max=10, #center={"lat": 36.978, "lon": -72.249},
                        zoom=1,
                        #hoverinfo = "data",
                        hover_data=["Case-Fatality Ratio"],
                        hover_name='Regions'
                       )
fig.update_layout(
    title=f'COVID-19 Global Active cases by country. Date: {date}',
)
fig.data[0].update(hovertemplate= '<b>%{hovertext}</b><br>Case-Fatality Ratio=%{marker.color}<extra></extra>')
fig.show()

In [15]:
write_json(fig, "case-fatality.json")

In [10]:
lat = df.Lat.unique()
long = df.Long_.unique()
(sum(lat)/len(lat),sum(long)/len(long))

(35.97824789303846, -72.24951081998732)

In [19]:
daily_us_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/11-19-2020.csv'
df = pd.read_csv(daily_us_url)
df = df.dropna(axis=0, subset=['Lat'])
df = df.fillna(0)
df['Active'] = df['Active'].astype(int)
date, time = df.iloc[0, 2].split()
fig = px.scatter_mapbox(df, lat="Lat", lon="Long_",
                        color="Active",
                        size="Active",
                        color_continuous_scale=px.colors.sequential.Jet,
                        size_max=40,
                        zoom=3, 
                        center={"lat": 38.5266, "lon": -96.72},              
                        hover_data=['Confirmed', 'Deaths', 'Recovered'],
                        hover_name='Province_State'
                       )
fig.update_layout(
    title=f'COVID-19 Active cases in USA by states, Date: {date}',
)
fig.data[0].update(hovertemplate='<b>%{hovertext}</b><br><br>Active=%{marker.color}<br>Confirmed=%{customdata[0]}<br>Deaths=%{customdata[1]}<br>Recovered=%{customdata[2]}<extra></extra>')
fig.show()

In [20]:
write_json(fig, "us.json")

In [95]:
df.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Alabama,US,2020-11-08 05:30:26,32.3182,-86.9023,202482,3082,84471.0,114929,1.0,4129.601473,1396368.0,0.0,1.522111,84000001,USA,28478.794906,0.0
1,Alaska,US,2020-11-08 05:30:26,61.3707,-152.4044,19306,84,7157.0,12065,2.0,2639.072101,747288.0,0.0,0.435098,84000002,USA,102152.020723,0.0
2,American Samoa,US,2020-11-08 05:30:26,-14.2710,-170.1320,0,0,0.0,0,60.0,0.000000,1768.0,0.0,0.000000,16,ASM,3177.512985,0.0
3,Arizona,US,2020-11-08 05:30:26,33.7298,-111.4312,257384,6147,42950.0,208287,4.0,3536.117698,1858835.0,0.0,2.388260,84000004,USA,25537.948515,0.0
4,Arkansas,US,2020-11-08 05:30:26,34.9697,-92.3731,120828,2068,106594.0,12166,5.0,4003.838553,1418493.0,0.0,1.711524,84000005,USA,47004.146061,0.0


In [22]:
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}
indexNames = df[ df['ISO3'] != 'USA' ].index 
df.drop(indexNames , inplace=True)
df = df.dropna(axis=0, subset=['Lat'])
df['Province_State'] = df['Province_State'].map(us_state_abbrev).fillna(df['Province_State'])

fig = go.Figure(data=go.Choropleth(
    locations=df['Province_State'], # Spatial coordinates
    z = df['Deaths'].astype(int), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
#     colorbar_title = "Millions USD",
))

fig.update_layout(
    title_text = f'COVID-19 Deaths by State, Date: {date}',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [24]:
write_json(fig, "death.json")

In [5]:
all_df = pd.read_csv('all_daily_us_until_1108.csv')
px.choropleth(all_df, 
              locations = 'Province_State',
              color="Active_scale", 
              animation_frame="Date",
              color_continuous_scale="orrd",
              locationmode='USA-states',
              scope="usa",
              title='COVID-19 Daily Active Cases per 1000 people by states',
              height=600
             )

In [6]:
#add some date range
all_df['Date_datetype'] = pd.to_datetime(all_df['Date'])
def date_range(start_date, end_date):
    return (all_df['Date_datetype']>start_date) & (all_df['Date_datetype']<end_date)
mask = date_range('2020-09-30', '2020-11-03')
px.choropleth(all_df.loc[mask], 
              locations = 'Province_State',
              color="Confirmed", 
              animation_frame="Date",
              color_continuous_scale="orrd",
              locationmode='USA-states',
              scope="usa",
              #range_color=(0, 20),
              title='COVID-19 Daily Active Cases people by states',
              height=600
             )